In [1]:
from imutils import paths
import numpy as np
import imutils
import cv2

In [2]:
rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (13, 5))
sqKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (21, 21))

In [3]:
img_path = '/media/hellrazer/ezedox/pdf files/images/passport/image3.jpg'
image = cv2.imread(img_path)
image = imutils.resize(image,height=600)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [4]:
gray = cv2.GaussianBlur(gray,(3,3),0)
blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT,rectKernel)

In [5]:
# cv2.imshow('After Image' , blackhat)
# cv2.waitKey(0)

In [6]:
# compute the Scharr gradient of the blackhat image and scale the
# result into the range [0, 255]
gradX = cv2.Sobel(blackhat, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
gradX = np.absolute(gradX)
(minVal, maxVal) = (np.min(gradX), np.max(gradX))
gradX = (255 * ((gradX - minVal) / (maxVal - minVal))).astype("uint8")


In [7]:
# cv2.imshow('After Image' , blackhat)
# cv2.waitKey(0)

In [8]:
# apply a closing operation using the rectangular kernel to close
	# gaps in between letters -- then apply Otsu's thresholding method
gradX = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE , rectKernel)
thresh = cv2.threshold(gradX , 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

In [9]:
thresh = cv2.morphologyEx(thresh , cv2.MORPH_CLOSE, sqKernel)
thresh = cv2.erode(thresh, None , iterations=4)

In [10]:
p = int(image.shape[1]*0.5)
thresh[:,0:p] = 0
thresh[:,image.shape[1] - p:] = 0

In [11]:
# find contours in the thresholded image and sort them by their
	# size
cnts = cv2.findContours(thresh.copy() , cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_SIMPLE)[-2]
cnts = sorted(cnts , key= cv2.contourArea, reverse=True)
for c in cnts:
# compute the bounding box of the contour and use the contour to
# compute the aspect ratio and coverage ratio of the bounding box
# width to the width of the image
    (x, y, w, h) = cv2.boundingRect(c)
    ar =w / float(gray.shape[1])
    crWidth = w/ float(gray.shape[1])
# check to see if the aspect ratio and coverage width are within
# acceptable criteria
    if ar>5 and crWidth > 0.75:
        pX = int((x + w) * 0.03)
        pY= int((y + h) * 0.03)
        (x, y) = (x - pX, y - pY)
        (w, h) = (w + (pX *2) , h + (pY+2))
        roi = image[y:y+h , x:x+w].copy()
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        break

In [12]:
cv2.imshow('Image' , image)
cv2.imshow('ROI' , roi)
cv2.waitKey(0)

NameError: name 'roi' is not defined